In [ ]:
import pickle
import numpy as np
from PIL import Image
import csv

In [ ]:
with open('datasets/colorferet_sketch_to_img_map.pickle', 'rb') as f:
    colorferet_map = pickle.load(f)

In [ ]:
i = 0
for k, v in colorferet_map.items():
    print(k, v)
    i += 1
    if i > 5:
        break

In [ ]:
# Testing that you can transform one triangle into another
fiducial = np.array([[312, 450, 387], 
                     [348, 352, 497]])
fiducial_3d = np.array([[312, 450, 387], 
                        [348, 352, 497], 
                        [1, 1, 1]])
goal = np.array([[300, 420, 360], 
                 [350, 250, 500], 
                 [0, 0, 0]])
w = goal @ np.linalg.inv(fiducial_3d)
mat = w[:2, :2]
translation = w[:2, 2:3]
mat @ fiducial + translation

In [ ]:
# Expects fiducial_points is 3 x 2 numpy array
def align_face(img, fiducial_points, point_targets=None, output_size=512):
    orig_width, orig_height = img.size
    if point_targets is None:
        point_targets = np.array([[0.4, 0.6, 0.5], [0.5, 0.5, 0.72]]) * output_size
    # Pupil 1: (0.4, 0.5), Pupil 2: (0.6, 0.5), Mouth: (0.5, 0.7)
    
    fiducial_3d = np.vstack((fiducial_points.T, np.ones((1, 3))))
    transform = np.vstack((point_targets @ np.linalg.inv(fiducial_3d), np.array([0, 0, 1])))
    transform_inv = np.linalg.inv(transform)

    data = transform_inv.flatten()[:6]
    return img.transform((output_size, output_size), Image.AFFINE, data)

In [ ]:
img1 = Image.open('datasets/colorferet/original_sketch/00001.jpg')
fiducial_points = np.zeros((3, 2))
with open('datasets/colorferet/sketch_points/00001.3pts', 'r') as f:
    reader = csv.reader(f, delimiter= ' ')
    for i, row in enumerate(reader):
        fiducial_points[i] = row

In [ ]:
align_face(img1, fiducial_points)